In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [2]:
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

In [3]:
num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

In [4]:
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

In [7]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size]) # (1,6,5) -> (6,5)
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

In [10]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))


(0, 'loss:', 1.6435561, 'prediction: ', array([[3, 3, 3, 3, 3, 3]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'llllll')
(1, 'loss:', 1.5243043, 'prediction: ', array([[3, 3, 3, 3, 3, 3]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'llllll')
(2, 'loss:', 1.4513379, 'prediction: ', array([[3, 3, 3, 3, 3, 3]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'llllll')
(3, 'loss:', 1.3871799, 'prediction: ', array([[3, 3, 3, 3, 3, 3]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'llllll')
(4, 'loss:', 1.3142462, 'prediction: ', array([[3, 3, 3, 3, 3, 3]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'llllll')
(5, 'loss:', 1.220543, 'prediction: ', array([[3, 3, 3, 3, 3, 3]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'llllll')
(6, 'loss:', 1.1109241, 'prediction: ', array([[3, 0, 3, 3, 3, 4]]), 'true Y: ', [[1, 0, 2, 3, 3, 4]])
('\tPrediction str: ', 'lhlllo')
(7, 'loss:', 0.99846429, 'prediction: ', array([[